In [1]:
import pandas as pd
import numpy as np
import requests
import swifter
import re

In [26]:
# Read in 1.2 Million Spotify Songs dataset
spotify_1m = pd.read_csv('./Resources/tracks_features.csv')

In [27]:
# Strip brackets and single quotes from artist name
spotify_1m['artist'] = spotify_1m['artists'].str.strip("[']")

# Preview dataframe
spotify_1m.head()

,id,name,album,album_id,artists,artist_ids,track_number,disc_number,explicit,danceability,...,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,year,release_date,artist
0,7lmeHLHBe4nmXzuXc0HDjk,Testify,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],['2d0hyoQ5ynDBnkvAbJKORj'],1,1,False,0.470,...,0.02610,0.000011,0.3560,0.503,117.906,210133,4.0,1999,1999-11-02,Rage Against The Machine
1,1wsRitfRRtWyEapl0q22o8,Guerrilla Radio,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],['2d0hyoQ5ynDBnkvAbJKORj'],2,1,True,0.599,...,0.01290,0.000071,0.1550,0.489,103.680,206200,4.0,1999,1999-11-02,Rage Against The Machine
2,1hR0fIFK2qRG3f3RF70pb7,Calm Like a Bomb,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],['2d0hyoQ5ynDBnkvAbJKORj'],3,1,False,0.315,...,0.02340,0.000002,0.1220,0.370,149.749,298893,4.0,1999,1999-11-02,Rage Against The Machine
3,2lbASgTSoDO7MTuLAXlTW0,Mic Check,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],['2d0hyoQ5ynDBnkvAbJKORj'],4,1,True,0.440,...,0.16300,0.000004,0.1210,0.574,96.752,213640,4.0,1999,1999-11-02,Rage Against The Machine
4,1MQTmpYOZ6fcMQc56Hdo7T,Sleep Now In the Fire,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],['2d0hyoQ5ynDBnkvAbJKORj'],5,1,False,0.426,...,0.00162,0.105000,0.0789,0.539,127.059,205600,4.0,1999,1999-11-02,Rage Against The Machine


In [53]:
# Read in Billboard Top 100 dataset
df = pd.read_csv('./Resources/charts.csv')

# Convert date column to date type
df['date'] = pd.to_datetime(df['date'])
# Create new column for year
df['year'] = df['date'].dt.year
# Remove special characters from artist and song name
df['artist'] = df['artist'].str.replace(r"\(.*\)","")
# df['song'] = df['song'].str.replace(r"\(.*\)","")

df['song'] = df['song'].str.replace("\"", "") \
                        # .str.replace(",", "") \
                        # .str.replace("!", "")


# Create new column showing number of weeks in #1 spot (if exists)
# df['weeks-at-no1'] = df[df['rank']==1].groupby(['song', 'artist'], as_index=False).count()['date']

# Remove single quotes/apostrophes from song names
df['song'] = df['song'].str.replace("'", "")

# Preview dataframe
df.head()

,date,rank,song,artist,last-week,peak-rank,weeks-on-board,year
0,2021-11-06,1,Easy On Me,Adele,1.0,1,3,2021
1,2021-11-06,2,Stay,The Kid LAROI & Justin Bieber,2.0,1,16,2021
2,2021-11-06,3,Industry Baby,Lil Nas X & Jack Harlow,3.0,1,14,2021
3,2021-11-06,4,Fancy Like,Walker Hayes,4.0,3,19,2021
4,2021-11-06,5,Bad Habits,Ed Sheeran,5.0,2,18,2021


In [46]:
# Create new dataframe of all unique Billboard charting songs
unique_billboard_tracks_df = df.groupby(['song','artist', 'year'], as_index=False).agg({'peak-rank': 'min', 
                                                                                        'weeks-on-board': 'max',
                                                                                        'weeks-at-no1': 'max'})
# unique_billboard_tracks_df['weeks-at-no1'] = df[df['rank']==1].groupby(['song', 'artist'], as_index=False).count()['date']

# Create separate dataframes for each decade
billboard_1960s = unique_billboard_tracks_df[(unique_billboard_tracks_df['year']<1970) & (unique_billboard_tracks_df['year']>=1960)]
billboard_1970s = unique_billboard_tracks_df[(unique_billboard_tracks_df['year']<1980) & (unique_billboard_tracks_df['year']>=1970)]
billboard_1980s = unique_billboard_tracks_df[(unique_billboard_tracks_df['year']<1990) & (unique_billboard_tracks_df['year']>=1980)]
billboard_1990s = unique_billboard_tracks_df[(unique_billboard_tracks_df['year']<2000) & (unique_billboard_tracks_df['year']>=1990)]
billboard_2000s = unique_billboard_tracks_df[(unique_billboard_tracks_df['year']<2010) & (unique_billboard_tracks_df['year']>=2000)]
billboard_2010s = unique_billboard_tracks_df[(unique_billboard_tracks_df['year']<2020) & (unique_billboard_tracks_df['year']>=2010)]
billboard_2020s = unique_billboard_tracks_df[unique_billboard_tracks_df['year']>=2020]

# Create list of dataframes
billboard_dfs = [billboard_1960s, billboard_1970s, billboard_1980s, billboard_1990s, billboard_2000s, billboard_2010s, billboard_2020s]

# Display dataframe
unique_billboard_tracks_df.head()
unique_billboard_tracks_df[unique_billboard_tracks_df['weeks-at-no1'].isnull()==False]

# df[(df['artist']=='Beastie Boys') & (df['year']==1986)]

,song,artist,year,peak-rank,weeks-on-board,weeks-at-no1
388,2055,Sleepy Hallow,2021,51,15,14.0
399,24,Kanye West,2021,43,1,2.0
481,50 Shots,YoungBoy Never Broke Again,2021,59,1,5.0
488,5500 Degrees,"EST Gee Featuring Lil Baby, 42 Dugg & Rylo Rod...",2021,92,2,4.0
521,7am On Bridle Path,Drake,2021,16,3,2.0
...,...,...,...,...,...,...
35538,You Right,Doja Cat & The Weeknd,2021,11,18,7.0
35565,You Should Probably Leave,Chris Stapleton,2021,40,17,8.0
35599,You Time,Scotty McCreery,2021,50,12,14.0
35741,Your Heart,Joyner Lucas & J. Cole,2021,32,3,7.0


In [2]:
# SPOTIFY API
# Import dependencies for Spotipy
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# Import Client ID and Client Secret
from config import cid, secret


# Create objects for accessing Spotify API
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [3]:
def search_spotify(song_title, artist):
    try:
        # print(f"Trying '{song_title}' by {artist}...")
        searchResults = sp.search(q=f"artist:{artist} track:{song_title}", type="track")
        track_id = searchResults['tracks']['items'][0]['id']
        # print(f"{song_title} by {artist} was successful!")
        # print(track_id)
        return track_id
    except:
        pass
        # print(f"'{song_title}' by {artist} returned 0 results.")

In [4]:
def adjust_parens(song_title):
    combinations = []

    strip_parens = ' '.join(song_title.strip(')(').split(')'))
    strip_parens = ' '.join(strip_parens.strip(' (').split('(')).replace('  ',' ')
    combinations.append(strip_parens)

    try:
        inside_parens = re.findall(r'\(.*?\)', song_title)[0].strip('()').strip()
    except IndexError:
        pass
        
    try:
        left_parens = song_title.split(f"({inside_parens})")[0].strip()
        if (left_parens not in combinations) & (len(left_parens) > 0): 
            combinations.append(left_parens)
    except:
        left_parens = "NA"

    try:
        right_parens = song_title.split(f"({inside_parens})")[1].strip()
        if (right_parens not in combinations) & (len(right_parens) > 0): 
            combinations.append(right_parens)
    except:
        right_parens = "NA"

    return combinations

In [5]:
# Create function to retrieve track ID from Spotify given the artist and song title
def get_track_id(song_title, artist):
    while True:
        # Select first artist if multiple listed with "Featuring" keyword
        if 'Featuring' in artist:
            artist = artist.split(' Featuring ')[0]
        # Select first artist if multiple listed with "with", "With", or "," substrings
        elif ' with ' in artist:
            artist = artist.split(' with ')[0]
        elif ' With ' in artist:
            artist = artist.split(' With ')[0]
        elif "," in artist:
            artist = artist.split(',')[0]

        # Make initial API search, return ID string if found
        found_id = search_spotify(song_title, artist)
        if found_id:
            return found_id

        # Search artist and song title (replacing words ending in "in" to "ing")
        song_title = re.sub(r"in\b", 'ing ', song_title)
        found_id = search_spotify(song_title, artist)
        if found_id:
            return found_id

        if '(' in song_title:
            for item in adjust_parens(song_title):
                found_id = search_spotify(item, artist)
                if found_id:
                    return found_id
                
        # Check for '/' character in song_title
        if '/' in song_title:
            # Try string on left side of '/'
            song_title = song_title.split('/')[0]
            found_id = search_spotify(song_title, artist)
            if found_id:
                return found_id

            # Try string on right side of '/'
            try:
                song_title = song_title.split('/')[1]
                found_id = search_spotify(song_title, artist)
                if found_id:
                    return found_id
            except:
                pass

        # Check for '&' character in artist name
        if ' & ' in artist:
            artist = artist.split(' & ')[0]
            found_id = search_spotify(song_title, artist)
            if found_id:
                return found_id
        # Check for 'X' character in artist name
        if ' X ' in artist:
            artist = artist.split(' X ')[0]
            found_id = search_spotify(song_title, artist)
            if found_id:
                return found_id
        # Check for 'x' character in artist name
        if ' x ' in artist:
            artist = artist.split(' x ')[0]
            found_id = search_spotify(song_title, artist)
            if found_id:
                return found_id
            
        # Print song title and artist for non-match
        if found_id:
            return found_id
        
        break

            # print(f"No ID found for '{song_title}' by {artist}")
    

In [6]:
# Returns tuple of audio features from Spotify for specified track_id
def get_audio_features(id):
    try:
        search_results = sp.audio_features(id)[0]
        danceability = search_results['danceability']
        energy = search_results['energy']
        key = search_results['key']
        loudness = search_results['loudness']
        mode = search_results['mode']
        speechiness = search_results['speechiness']
        acousticness = search_results['acousticness']
        instrumentalness = search_results['instrumentalness']
        liveness = search_results['liveness']
        valence = search_results['valence']
        tempo = search_results['tempo']
        duration_ms = search_results['duration_ms']
        time_signature = search_results['time_signature']
    except:
        return np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan

    return danceability, energy, key, loudness, mode, speechiness, acousticness, instrumentalness, liveness, \
                valence, tempo, duration_ms, time_signature

In [11]:
# Add new "track_id" column for 1960's dataset based on Spotify API calls and export as csv
billboard_1960s['track_id'] = billboard_1960s[['song', 'artist']].swifter.apply(lambda row:get_track_id(row.song,row.artist),axis=1)
billboard_1960s.to_csv(f'./Resources/billboard_1960s.csv', index=False)

Pandas Apply:   0%|          | 0/7792 [00:00<?, ?it/s]

/Users/glena.dagger/Downloads/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [64]:
# Add new "track_id" column for 1970's dataset based on Spotify API calls and export as csv
billboard_1970s['track_id'] = billboard_1970s[['song', 'artist']].swifter.apply(lambda row:get_track_id(row.song,row.artist),axis=1)
billboard_1970s.to_csv(f'./Resources/billboard_1970s.csv', index=False)

Pandas Apply:   0%|          | 0/6251 [00:00<?, ?it/s]

/Users/glena.dagger/Downloads/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [13]:
# Add new "track_id" column for 1980's dataset based on Spotify API calls and export as csv
billboard_1980s['track_id'] = billboard_1980s[['song', 'artist']].swifter.apply(lambda row:get_track_id(row.song,row.artist),axis=1)
billboard_1980s.to_csv(f'./Resources/billboard_1980s.csv', index=False)

Pandas Apply:   0%|          | 0/5126 [00:00<?, ?it/s]

/Users/glena.dagger/Downloads/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [14]:
# Add new "track_id" column for 1990's dataset based on Spotify API calls and export as csv
billboard_1990s['track_id'] = billboard_1990s[['song', 'artist']].swifter.apply(lambda row:get_track_id(row.song,row.artist),axis=1)
billboard_1990s.to_csv(f'./Resources/billboard_1990s.csv', index=False)

Pandas Apply:   0%|          | 0/4405 [00:00<?, ?it/s]

/Users/glena.dagger/Downloads/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [13]:
# Add new "track_id" column for 2000's dataset based on Spotify API calls and export as csv
billboard_2000s['track_id'] = billboard_2000s[['song', 'artist']].swifter.apply(lambda row:get_track_id(row.song,row.artist),axis=1)
billboard_2000s.to_csv(f'./Resources/billboard_2000s.csv', index=False)

Pandas Apply:   0%|          | 0/4433 [00:00<?, ?it/s]

/Users/glena.dagger/Downloads/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [13]:
# Add new "track_id" column for 2010's dataset based on Spotify API calls and export as csv
billboard_2010s['track_id'] = billboard_2010s[['song', 'artist']].swifter.apply(lambda row:get_track_id(row.song,row.artist),axis=1)
billboard_2010s.to_csv(f'./Resources/billboard_2010s.csv', index=False)

Pandas Apply:   0%|          | 0/5582 [00:00<?, ?it/s]

/Users/glena.dagger/Downloads/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [12]:
# Add new "track_id" column for 2020's dataset based on Spotify API calls and export as csv
billboard_2020s['track_id'] = billboard_2020s[['song', 'artist']].swifter.apply(lambda row:get_track_id(row.song,row.artist),axis=1)
billboard_2020s.to_csv(f'./Resources/billboard_2020s.csv', index=False)

Pandas Apply:   0%|          | 0/1479 [00:00<?, ?it/s]

/Users/glena.dagger/Downloads/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [78]:
# Read in each decade csv file for Billboard data
df_1960s = pd.read_csv('./Resources/billboard_1960s.csv')
df_1970s = pd.read_csv('./Resources/billboard_1970s.csv')
df_1980s = pd.read_csv('./Resources/billboard_1980s.csv')
df_1990s = pd.read_csv('./Resources/billboard_1990s.csv')
df_2000s = pd.read_csv('./Resources/billboard_2000s.csv')
df_2010s = pd.read_csv('./Resources/billboard_2010s.csv')
df_2020s = pd.read_csv('./Resources/billboard_2020s.csv')

In [9]:
all_billboard_df = pd.concat([df_1960s, df_1970s, df_1980s, df_1990s, df_2000s, df_2010s, df_2020s])
all_billboard_df.to_csv('./Resources/billboard_ALL.csv', index=False)

In [10]:
# Get audio features for each dataframe and export as separate csv
def add_features_to_df(df, file_name):
    df.dropna(subset=['track_id'], inplace=True)
    df['danceability'], df['energy'], df['key'], df['loudness'], df['mode'], df['speechiness'], df['acousticness'], \
    df['instrumentalness'], df['liveness'], df['valence'], df['tempo'], df['duration_ms'], df['time_signature'] = \
                                          zip(*df['track_id'].map(get_audio_features))

    df.to_csv(f'./Resources/{file_name}_features.csv', index=False)

In [7]:
# Read in each decade csv file for Billboard data
df_1960s = pd.read_csv('./Resources/billboard_1960s_features.csv')
df_1970s = pd.read_csv('./Resources/billboard_1970s_features.csv')
df_1980s = pd.read_csv('./Resources/billboard_1980s_features.csv')
df_1990s = pd.read_csv('./Resources/billboard_1990s_features.csv')
df_2000s = pd.read_csv('./Resources/billboard_2000s_features.csv')
df_2010s = pd.read_csv('./Resources/billboard_2010s_features.csv')
df_2020s = pd.read_csv('./Resources/billboard_2020s_features.csv')

In [108]:
# Add feature columns to each decade dataframe and export as new csv file
add_features_to_df(df_2020s, "billboard_2020s")
add_features_to_df(df_2010s, "billboard_2010s")
add_features_to_df(df_2000s, "billboard_2000s")
add_features_to_df(df_1990s, "billboard_1990s")
add_features_to_df(df_1980s, "billboard_1980s")
add_features_to_df(df_1970s, "billboard_1970s")
add_features_to_df(df_1960s, "billboard_1960s")

In [18]:
# Combine all complete billboard dfs into one and export to csv file
all_billboard_features_df = pd.concat([df_1960s, df_1970s, df_1980s, df_1990s, df_2000s, df_2010s, df_2020s])
all_billboard_features_df.drop("weeks-at-no1", axis=1, inplace=True)
all_billboard_features_df.dropna(inplace=True)
all_billboard_features_df.to_csv('./Resources/billboard_ALL_features.csv', index=False)

In [38]:
all_billboard_features_df.groupby('year', as_index=False).median().sort_values(by="loudness", ascending=False)

,year,peak-rank,weeks-on-board,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
49,2009,47.0,10.0,0.5910,0.7530,5.0,-5.0520,1.0,0.04620,0.06130,0.000000,0.1420,0.4970,128.0420,228347.0,4.0
48,2008,45.0,12.0,0.6070,0.7460,5.0,-5.0960,1.0,0.05040,0.06830,0.000000,0.1350,0.5050,124.9680,231133.0,4.0
51,2011,53.0,7.0,0.6030,0.7520,6.0,-5.1145,1.0,0.05015,0.05875,0.000000,0.1355,0.4820,127.0665,223273.5,4.0
50,2010,48.0,8.0,0.6050,0.7580,5.0,-5.1550,1.0,0.04790,0.06610,0.000000,0.1300,0.5160,123.8490,227533.0,4.0
47,2007,42.0,14.0,0.6205,0.7355,5.0,-5.1820,1.0,0.05000,0.06810,0.000000,0.1340,0.5405,122.3775,231240.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9,1969,46.0,7.0,0.5310,0.5170,5.0,-10.5350,1.0,0.03610,0.39800,0.000024,0.1430,0.6800,114.9640,179493.0,4.0
28,1988,30.0,13.0,0.6545,0.7040,5.0,-10.6335,1.0,0.03740,0.10650,0.000084,0.1180,0.6775,118.4450,258040.0,4.0
13,1973,42.0,9.0,0.5510,0.5640,5.0,-10.7350,1.0,0.03910,0.37500,0.000067,0.1250,0.7070,120.0240,208227.0,4.0
12,1972,44.0,9.0,0.5410,0.5450,5.0,-10.7380,1.0,0.03760,0.41100,0.000058,0.1270,0.6470,119.9860,205480.0,4.0
